In [4]:
# Kütüphaneler

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale     # standardizasoyon işlemleri
from sklearn.preprocessing import StandardScaler    # standardizasoyon işlemleri
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from warnings import filterwarnings
filterwarnings("ignore")     # önceki bölümlerde çalıştırdığımızda çıkan gereksiz uyarı mesajları çıkmasın diye

In [5]:
df = pd.read_csv("C:\python_py\Makine Öğrenmesi-2\Hitters.csv")
df = df.dropna()

dms = pd.get_dummies(df[["League" , "Division" , "NewLeague"]])

y = df["Salary"]
x___ = df.drop(["Salary" , "League" , "Division" , "NewLeague"] , axis=1).astype("float64")
x = pd.concat([x___ , dms[["League_N" , "Division_W" , "NewLeague_N"]]] , axis=1)

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25 , random_state=42)

In [6]:
x_train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
183,328.0,91.0,12.0,51.0,43.0,33.0,2.0,342.0,94.0,12.0,51.0,44.0,33.0,145.0,59.0,8.0,True,False,True
229,514.0,144.0,0.0,67.0,54.0,79.0,9.0,4739.0,1169.0,13.0,583.0,374.0,528.0,229.0,453.0,15.0,True,False,True
286,593.0,152.0,23.0,69.0,75.0,53.0,6.0,2765.0,686.0,133.0,369.0,384.0,321.0,315.0,10.0,6.0,False,True,False
102,233.0,49.0,2.0,41.0,23.0,18.0,8.0,1350.0,336.0,7.0,166.0,122.0,106.0,102.0,132.0,10.0,False,False,False
153,341.0,95.0,6.0,48.0,42.0,20.0,10.0,2964.0,808.0,81.0,379.0,428.0,221.0,158.0,4.0,5.0,True,True,True


# MODEL

In [7]:
knn_model = KNeighborsRegressor().fit(x_train , y_train)    # Modeli kurduk.

In [8]:
knn_model.n_neighbors   # K'yi (En yakın K tane gözlemi seçecek) default olarak 5 seçmiş

5

In [9]:
knn_model.metric    # Hangi uzaklık ölçme yöntemini kullanmış

'minkowski'

# TAHMİN

In [10]:
knn_model.predict(x_test)[0:5]

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ])

In [11]:
# Test setinin hata oranı

y_pred = knn_model.predict(x_test)

RMSE = np.sqrt(mean_squared_error(y_pred , y_test))
RMSE

426.6570764525201

# MODEL TUNİNG (DOĞRULAMA)

In [13]:
# "knn_model.n_neighbors" değerinin farklı değerler alıdığında nasıl sonuçlar alacağımızı el yordamıyla döngü yazarak ifade edelim. Daha sonra aynı işlemi GridSearchCV ile fonksiyonel şekilde gösterelim

RMSE = []

for k in range(10):
    k = k + 1   # 0-9 arasında değil de 1-10 arasındaki değerleri versin diye başa yazdım
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(x_train , y_train)
    y_pred = knn_model.predict(x_test)  # x_train de yazabilirdik
    rmse = np.sqrt(mean_squared_error(y_pred , y_test))
    RMSE.append(rmse)
    print("k=" , k , "için RMSE değeri ->" , rmse)

# SONUCA GÖRE 1-10 arasındaki K DEĞERLERİNDEN EN İDEALi K=8 

k= 1 için RMSE değeri -> 455.03925390751965
k= 2 için RMSE değeri -> 415.99629571490965
k= 3 için RMSE değeri -> 420.6765370082348
k= 4 için RMSE değeri -> 428.8564674588792
k= 5 için RMSE değeri -> 426.6570764525201
k= 6 için RMSE değeri -> 423.5071669008732
k= 7 için RMSE değeri -> 414.9361222421057
k= 8 için RMSE değeri -> 413.7094731463598
k= 9 için RMSE değeri -> 417.84419990871265
k= 10 için RMSE değeri -> 421.6252180741266


In [15]:
# Aynı işlemi "GridSearhCV" ile fonksiyonel şekilde gösterelim. GridSearhCV -> kullanacak olduğüumuz makine öğrenmesi algoritmalarında belirlemeye çalıştığımız hiperparametrelerin (attribute'ların) değerlerini belirlemek için
                                                                             # kullanılan bir fonksiyondur.

knn_params = {"n_neighbors":np.arange(1,30,1)}

knn = KNeighborsRegressor()

knn_cv_model = GridSearchCV(knn , knn_params , cv=10).fit(x_train , y_train)

knn_cv_model.best_params_   # ÜSTTEKİYLE AYNI SONUCU BULDUK -> SONUCA GÖRE 1-10 arasındaki K DEĞERLERİNDEN EN İDEALi K=8 

# El yordamıyla hesapladığımız değeri GridSearhCV fonksiyonu ile hesaplamış olduk. Genelde bu yöntemi kullanmamız gerekmekte.

{'n_neighbors': 8}

In [16]:
# FİNAL MODEL

knn_tuned = KNeighborsRegressor(n_neighbors=knn_cv_model.best_params_["n_neighbors"]).fit(x_train , y_train)

In [18]:
# FİNAL MODEL'imizin hata oranını ölçelim

y_pred = knn_tuned.predict(x_test)

RMSE = np.sqrt(mean_squared_error(y_pred , y_test))
RMSE

# İLK BAŞTAKI TEST HATAMIZ 426'idi. Görüldüğü üzere hata oranımız düştü.

413.7094731463598